In [1]:
import pandas as pd
import pickle
import numpy as np

In [46]:
output = pickle.load(open('dataset_level2.pkl','rb'))

In [47]:
output.isnull().sum()

match_id               0
batting_team           0
bowling_team           0
ball                   0
runs                   0
player_dismissed       0
city                8423
venue                  0
dtype: int64

In [48]:
output[output['city'].isnull()]['venue'].value_counts()

Dubai International Cricket Stadium        2966
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sylhet International Cricket Stadium        128
Sharjah Cricket Stadium                     127
Carrara Oval                                 64
Name: venue, dtype: int64

In [49]:
cities = np.where(output['city'].isnull(),output['venue'].str.split().apply(lambda x:x[0]),df['city'])

In [50]:
output['city'] = cities

In [51]:
output.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [53]:
total_df = output.groupby('match_id').sum()['runs'].reset_index()
output = output.merge(total_df,on='match_id')


In [54]:
output

,match_id,batting_team,bowling_team,ball,runs_x,player_dismissed,city,venue,runs_y
0,2,West Indies,India,0.1,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,95
1,2,West Indies,India,0.2,0,JD Campbell,Lauderhill,Central Broward Regional Park Stadium Turf Ground,95
2,2,West Indies,India,0.3,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,95
3,2,West Indies,India,0.4,1,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,95
4,2,West Indies,India,0.5,0,0,Lauderhill,Central Broward Regional Park Stadium Turf Ground,95
...,...,...,...,...,...,...,...,...,...
61931,964,Pakistan,New Zealand,19.2,1,0,Auckland,Eden Park,153
61932,964,Pakistan,New Zealand,19.3,0,Faheem Ashraf,Auckland,Eden Park,153
61933,964,Pakistan,New Zealand,19.4,2,0,Auckland,Eden Park,153
61934,964,Pakistan,New Zealand,19.5,1,0,Auckland,Eden Park,153


In [55]:
output['current_score'] = output.groupby('match_id').cumsum()['runs_x']


In [56]:
output['over'] = output['ball'].apply(lambda x:str(x).split(".")[0])
output['ball_no'] = output['ball'].apply(lambda x:str(x).split(".")[1])

In [57]:
output['balls_bowled'] = (output['over'].astype('int')*6) + output['ball_no'].astype('int')


In [58]:
output['crr'] = round((output['current_score']*6)/output['balls_bowled'],2)


In [59]:
output['player_dismissed'] = output['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
output['player_dismissed'] = output['player_dismissed'].astype('int')
output['player_dismissed'] = output.groupby('match_id').cumsum()['player_dismissed']
output['wickets_left'] = 10 - output['player_dismissed']


In [60]:
final_df = output[['match_id','batting_team','bowling_team','runs_x','current_score','balls_bowled','wickets_left','crr','city','runs_y']]


In [61]:
final_df


,match_id,batting_team,bowling_team,runs_x,current_score,balls_bowled,wickets_left,crr,city,runs_y
0,2,West Indies,India,0,0,1,10,0.00,Lauderhill,95
1,2,West Indies,India,0,0,2,9,0.00,Lauderhill,95
2,2,West Indies,India,0,0,3,9,0.00,Lauderhill,95
3,2,West Indies,India,1,1,4,9,1.50,Lauderhill,95
4,2,West Indies,India,0,1,5,9,1.20,Lauderhill,95
...,...,...,...,...,...,...,...,...,...,...
61931,964,Pakistan,New Zealand,1,149,116,2,7.71,Auckland,153
61932,964,Pakistan,New Zealand,0,149,117,1,7.64,Auckland,153
61933,964,Pakistan,New Zealand,2,151,118,1,7.68,Auckland,153
61934,964,Pakistan,New Zealand,1,152,119,1,7.66,Auckland,153


In [62]:
final_df = final_df.sample(final_df.shape[0])


In [63]:
final_df['balls_left'] = 120 - final_df['balls_bowled']
final_df['balls_left'] = final_df['balls_left'].apply(lambda x:0 if x<0 else x)


In [64]:
final_df['crr'] = round((final_df['current_score']*6)/final_df['balls_bowled'],2)


In [65]:
final_df.drop(columns=['balls_bowled'],inplace=True)


In [66]:
groups = final_df.groupby('match_id')


In [67]:
match_ids = final_df['match_id'].unique()
last_five = []
for id in match_ids:
    last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs_x'].values.tolist())

/var/folders/db/_plvhxn51ps9fymsypzjnfkh0000gn/T/ipykernel_29633/1152253557.py:4: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['batting_team', 'bowling_team', 'city'], dtype='object')
  last_five.extend(groups.get_group(id).rolling(window=30).sum()['runs_x'].values.tolist())


In [68]:
final_df['last_five'] = last_five


In [69]:
final_df


,match_id,batting_team,bowling_team,runs_x,current_score,wickets_left,crr,city,runs_y,balls_left,last_five
7986,139,South Africa,England,0,79,7,5.85,Southampton,142,39,NaN
4874,90,Pakistan,Australia,1,47,8,5.32,Colombo,149,67,NaN
24843,409,Pakistan,Bangladesh,1,21,10,11.45,Kolkata,201,109,NaN
20321,325,New Zealand,Sri Lanka,1,138,7,8.36,Mount Maunganui,182,21,NaN
29847,495,South Africa,Bangladesh,1,89,7,6.94,Mirpur,148,43,NaN
...,...,...,...,...,...,...,...,...,...,...,...
19306,314,Australia,England,0,56,7,6.22,Southampton,157,66,19.0
5141,99,Australia,England,0,87,5,6.78,Melbourne,147,43,20.0
24141,399,Sri Lanka,India,0,53,8,8.83,Colombo,152,84,19.0
28428,479,Australia,Sri Lanka,4,16,10,9.60,Adelaide,233,110,NaN


In [70]:
final_df.dropna(inplace=True)


In [71]:
final_df

,match_id,batting_team,bowling_team,runs_x,current_score,wickets_left,crr,city,runs_y,balls_left,last_five
60674,948,England,Pakistan,1,158,5,9.20,London,185,17,32.0
17913,293,Sri Lanka,New Zealand,0,5,10,7.50,Kandy,125,116,32.0
8913,154,England,Pakistan,4,40,7,6.67,Dubai,160,84,31.0
33955,557,South Africa,Pakistan,1,153,7,8.42,Johannesburg,188,11,31.0
13182,228,Sri Lanka,Pakistan,2,112,7,7.91,London,150,35,31.0
...,...,...,...,...,...,...,...,...,...,...,...
20375,326,Australia,West Indies,0,26,7,5.57,London,169,92,21.0
13451,230,Pakistan,South Africa,1,98,2,5.71,Abu Dhabi,119,17,20.0
19306,314,Australia,England,0,56,7,6.22,Southampton,157,66,19.0
5141,99,Australia,England,0,87,5,6.78,Melbourne,147,43,20.0


In [72]:
eligible_cities = final_df['city'].value_counts()[final_df['city'].value_counts() > 600].index.tolist()
final_df = final_df[final_df['city'].isin(eligible_cities)]

In [73]:
X = final_df.drop(columns=['match_id','runs_x','runs_y'])
y = final_df['runs_y']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
X_test

,batting_team,bowling_team,current_score,wickets_left,crr,city,balls_left,last_five
38984,Sri Lanka,Bangladesh,97,5,6.61,Colombo,32,49.0
38544,India,New Zealand,59,7,7.22,Wellington,71,46.0
39425,South Africa,Pakistan,43,9,7.82,Dubai,87,43.0
24962,West Indies,South Africa,10,10,7.50,Johannesburg,112,40.0
22424,South Africa,Pakistan,172,7,9.92,Cape Town,16,22.0
...,...,...,...,...,...,...,...,...
12246,New Zealand,Bangladesh,19,10,5.70,Pallekele,100,11.0
51910,Australia,England,90,6,6.21,Barbados,33,38.0
9423,South Africa,England,117,8,12.32,Mumbai,63,39.0
38206,England,West Indies,101,8,8.08,Mumbai,45,44.0


In [75]:
!pip install xgboost


     |████████████████████████████████| 1.7 MB 1.1 MB/s eta 0:00:01


In [76]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score,mean_absolute_error

trf = ColumnTransformer([
    ('trf',OneHotEncoder(sparse=False,drop='first'),['batting_team','bowling_team','city'])
]
,remainder='passthrough')


In [77]:
pipe = Pipeline(steps=[
    ('step1',trf),
    ('step2',StandardScaler()),
    ('step3',XGBRegressor(n_estimators=1000,learning_rate=0.2,max_depth=12,random_state=1))
])

In [78]:
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_absolute_error(y_test,y_pred))

0.9430460737696232
3.2908248921955376


In [83]:
pickle.dump(pipe,open('pipe.pk1','wb'))